## Imports

In [1]:
import tensorflow as tf
import keras
import json
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import GridSearchCV

## Constants

In [2]:
# Constantes
IS_TO_FIND_BEST_HYPERPARAMS = True
IS_TO_TRAIN = True
IS_TO_REPLICATE = True

BATCH_SIZE = 64
IMG_HEIGHT = 256
IMG_WIDTH = 256
LEARNING_RATE = 1e-5

DATASET_PATH = "./cats_and_dogs"
DATASET_TRAIN_PATH = f"{DATASET_PATH}/train"
DATASET_VAL_PATH = f"{DATASET_PATH}/validation"

NUM_CLASSES = 2

SEED = 7654321

VAL_TEST_RATIO = 0.5

MAX_EPOCHS = 100

WEIGHTS_FILE_EXT = "weights.h5"
HYPERPARAMS_FILE_EXT = "hyperparams.json"

# callback para parar o treino caso não se verifiquem melhorias na loss
EARLY_STOPPING = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

INITIALIZER = keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=SEED)
GLOROT_UNIFORM_INITIALIZER = keras.initializers.GlorotUniform(seed=SEED)

DEFAULT_LOSS = "binary_crossentropy"
DEFAULT_OPTIMIZER = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
DEFAULT_METRICS = ["accuracy", keras.metrics.F1Score]

2025-03-24 09:33:39.470367: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-03-24 09:33:39.470539: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-03-24 09:33:39.470543: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1742808819.470841   13229 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1742808819.470985   13229 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Dataset load

In [3]:
train_ds = keras.utils.image_dataset_from_directory(
    DATASET_TRAIN_PATH,
    labels="inferred",
    label_mode="binary",
    seed=SEED,
    batch_size=BATCH_SIZE,
    image_size=(IMG_WIDTH, IMG_HEIGHT),
    verbose=False,
)

val_ds, test_ds = keras.utils.image_dataset_from_directory(
    DATASET_VAL_PATH,
    labels="inferred",
    label_mode="binary",
    validation_split=VAL_TEST_RATIO,
    subset="both",
    seed=SEED,
    image_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    verbose=False,
)

# load the datasets into memory - once loaded, the order of the batches will no longer change
train_ds = train_ds.cache()
val_ds = val_ds.cache()
test_ds = test_ds.cache()

Using 500 files for training.
Using 500 files for validation.


Challenge: Find the best architecture that can generalize very well the problem features, and classify each animal with minimal error.

* Higher number of convolutional layers will produce more feature extraction from the training dataset, it can indicate a tendency to overfit the model.
* Higher number of pooling layers will subsample the image over the original size, the model will not extract correctly the differences in each class for each feature.
* ...

rephrase these topics.

Pressupostos:
* Número de filtros em cada camada: 32 x degrau do nível hierarquico, ex: 1º nível 32, 2º nível 64, etc. (explicar definição de nível hierárquico)
* Pooling technique: Max Pooling

### Procedure to define the best model considering the above assumptions:

Identify:
1. the best CNN architecture
2. the best filter size
3. best pooling size
4. the dense layer number of neurons
5. data augmentation hyperparameters

ref: https://github.com/dnouri/nolearn/blob/master/docs/notebooks/CNN_tutorial.ipynb

### Finding the best custom CNN architecture

#### Model with two levels and higher emphasis on the lower level features

In [4]:
low_level_feats_model = keras.Sequential(
    [
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

        # Low level features
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        # High level features
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        # Hidden layer
        layers.Dense(96, activation="relu", kernel_initializer=INITIALIZER),
        layers.Dense(128, activation="relu", kernel_initializer=INITIALIZER),

        # Classifier
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid", kernel_initializer=INITIALIZER)
    ], name="low_level_feats_model"
)

low_level_feats_model.compile(
    loss=DEFAULT_LOSS,
    optimizer=DEFAULT_OPTIMIZER,
    metrics=DEFAULT_METRICS
)

In [5]:
low_level_feats_model.summary()

Model: "low_level_feats_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64, 64, 96)     │         6,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64, 64, 128)    │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 524288)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │       524,289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 691,681 (2.64 MB)

 Trainable params: 691,681 (2.64 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
low_level_feats_model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=f"models/{low_level_feats_model.name}.{WEIGHTS_FILE_EXT}",
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
    save_best_only=True,
)

low_level_feats_model_history = low_level_feats_model.fit(
    train_ds,
    epochs=MAX_EPOCHS,
    validation_data=val_ds,
    callbacks=[low_level_feats_model_checkpoint, EARLY_STOPPING],
)

Epoch 1/100


2025-03-24 09:33:40.434311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 885ms/step - accuracy: 0.5098 - f1_score: 0.6532 - loss: 0.9454 - val_accuracy: 0.5000 - val_f1_score: 0.6649 - val_loss: 0.8642
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 881ms/step - accuracy: 0.5434 - f1_score: 0.6532 - loss: 0.8171 - val_accuracy: 0.5100 - val_f1_score: 0.6649 - val_loss: 0.7966
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 869ms/step - accuracy: 0.5549 - f1_score: 0.6532 - loss: 0.7607 - val_accuracy: 0.5120 - val_f1_score: 0.6649 - val_loss: 0.7657
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 861ms/step - accuracy: 0.5617 - f1_score: 0.6532 - loss: 0.7331 - val_accuracy: 0.5380 - val_f1_score: 0.6649 - val_loss: 0.7469
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 863ms/step - accuracy: 0.5629 - f1_score: 0.6532 - loss: 0.7144 - val_accuracy: 0.5420 - val_f1_score: 0.6649 - val_loss: 0.7343
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 870ms/step - accuracy: 0.5689 - f1_score: 0.6532 - loss: 0.7011 - val_accuracy: 0.5500 - val_f1_score: 0.6649

#### Model with a sequence of convolutional layer followed by pooling layer

In [7]:
conv_pool_seq_model = keras.Sequential(
    [
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        layers.Conv2D(96, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        layers.Conv2D(160, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        layers.Conv2D(192, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        layers.Conv2D(224, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        # Hidden layer
        layers.Dense(352, activation="relu", kernel_initializer=INITIALIZER),
        layers.Dense(384, activation="relu", kernel_initializer=INITIALIZER),

        # Classifier
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid", kernel_initializer=INITIALIZER)
    ], name="conv_pool_seq_model"
)
conv_pool_seq_model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=["accuracy", keras.metrics.F1Score]
)

In [ ]:
conv_pool_seq_model.summary()

Model: "conv_pool_seq_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 64, 64, 96)     │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 32, 32, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 32, 32, 128)    │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 16, 16, 160)    │       184,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 8, 8, 160)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 8, 8, 192)      │       276,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 4, 4, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 4, 4, 224)      │       387,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 2, 2, 224)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2, 2, 352)      │        79,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2, 2, 384)      │       135,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │         1,537 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,250,241 (4.77 MB)

 Trainable params: 1,250,241 (4.77 MB)

 Non-trainable params: 0 (0.00 B)

: 

In [ ]:
conv_pool_seq_model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=f"models/{conv_pool_seq_model.name}.{WEIGHTS_FILE_EXT}",
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
    save_best_only=True,
)

conv_pool_seq_model_history = conv_pool_seq_model.fit(
    train_ds,
    epochs=MAX_EPOCHS,
    validation_data=val_ds,
    callbacks=[conv_pool_seq_model_checkpoint, EARLY_STOPPING],
)

Epoch 1/100


#### Model with emphasis on three level features

In [ ]:
three_feature_levels_model = keras.Sequential(
    [
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

        # Low level features
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(pool_size=(4, 4)),

        # Mid level features
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        # High level features
        layers.Conv2D(96, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(96, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(96, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        # Hidden layer
        layers.Dense(128, activation="relu", kernel_initializer=INITIALIZER),
        layers.Dense(160, activation="relu", kernel_initializer=INITIALIZER),

        # Classifier
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid", kernel_initializer=INITIALIZER)
    ], name="three_feature_levels_model"
)

three_feature_levels_model.compile(
    loss=DEFAULT_LOSS,
    optimizer=DEFAULT_OPTIMIZER,
    metrics=DEFAULT_METRICS
)

In [ ]:
three_feature_levels_model.summary()

In [ ]:
three_feature_levels_model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=f"models/{three_feature_levels_model.name}.{WEIGHTS_FILE_EXT}",
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
    save_best_only=True,
)

three_feature_levels_model_history = three_feature_levels_model.fit(
    train_ds,
    epochs=MAX_EPOCHS,
    validation_data=val_ds,
    callbacks=[three_feature_levels_model_checkpoint, EARLY_STOPPING],
)

#### Model with a mixture of the second and third models

In [ ]:
mix_cnn_model = keras.Sequential(
    [
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

        # Low level features
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(pool_size=(4, 4)),

        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(pool_size=(4, 4)),

        # Mid level features
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        # High level features
        layers.Conv2D(96, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.Conv2D(96, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        layers.Conv2D(96, 3, padding="same", activation="relu", kernel_initializer=GLOROT_UNIFORM_INITIALIZER),
        layers.MaxPooling2D(),

        # Hidden layer
        layers.Dense(128, activation="relu", kernel_initializer=INITIALIZER),
        layers.Dense(160, activation="relu", kernel_initializer=INITIALIZER),

        # Classifier
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid", kernel_initializer=INITIALIZER)
    ], name="mix_cnn_model"
)

mix_cnn_model.compile(
    loss=DEFAULT_LOSS,
    optimizer=DEFAULT_OPTIMIZER,
    metrics=DEFAULT_METRICS
)

In [ ]:
mix_cnn_model.summary()

In [ ]:
mix_cnn_model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=f"models/{mix_cnn_model.name}.{WEIGHTS_FILE_EXT}",
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
    save_best_only=True,
)

mix_cnn_model_history = mix_cnn_model.fit(
    train_ds,
    epochs=MAX_EPOCHS,
    validation_data=val_ds,
    callbacks=[mix_cnn_model_checkpoint, EARLY_STOPPING],
)

#### Results

#### Discussion

### Find best pooling layers

### Find best CNN hyperparameters (data augmentation included)

In [ ]:
# initializer = keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=SEED)
# data_augmentation = keras.Sequential(
#     [layers.RandomRotation(0.2), layers.RandomZoom(0.15)]
# )

catOrDog_model = keras.Sequential(
    [
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        # data_augmentation,
        layers.Conv2D(8, 5, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(4, 4)),
        layers.Dropout(0.15),
        layers.Flatten(),
        layers.Dense(1024, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)
# definição do algoritmo de otimização e da função de perda (loss)
catOrDog_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# sumário do modelo
catOrDog_model.summary()

In [ ]:
# Make predictions
output_pred = catOrDog_model.predict(test_ds)
y_pred = (output_pred > 0.5).astype(int).flatten()  # Convert sigmoid output to 0 or 1

# Generate Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Display Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Cat", "Dog"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
history = catOrDog_model.fit(
    train_ds,
    epochs=MAX_EPOCHS,
    validation_data=val_ds,
    callbacks=[BEST_MODEL_CHECKPOINT],
)

In [ ]:
y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)

# Make predictions
output_pred = catOrDog_model.predict(val_ds)
y_pred = (output_pred > 0.5).astype(int).flatten()  # Convert sigmoid output to 0 or 1

# Generate Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Create subplots (1 row, 3 columns)
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot Accuracy
axes[0].plot(history.history["accuracy"], label="Train Accuracy")
axes[0].plot(history.history["val_accuracy"], label="Validation Accuracy")
axes[0].set_title("Model Accuracy")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Accuracy")
axes[0].legend()
axes[0].grid(True, linestyle="--")

# Plot Loss
axes[1].plot(history.history["loss"], label="Train Loss")
axes[1].plot(history.history["val_loss"], label="Validation Loss")
axes[1].set_title("Model Loss")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Loss")
axes[1].set_ylim(0, 2.0)
axes[1].legend()
axes[1].grid(True, linestyle="--")

# Plot Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Cat", "Dog"])
disp.plot(ax=axes[2], cmap=plt.cm.Blues)
axes[2].set_title("Confusion Matrix")

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier  # Updated import
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define a function to create the model
def create_model(learning_rate=0.001, dropout_rate=0.25, dense_units=1012):
    initializer = keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=SEED)

    model = keras.Sequential([
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        data_augmentation,
        layers.Conv2D(16, 5, padding='same', activation='relu', kernel_initializer=initializer),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Conv2D(256, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Conv2D(512, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(7, 7)),
        layers.Dropout(dropout_rate),
        layers.Flatten(),
        layers.Dense(dense_units, activation='relu', kernel_initializer=initializer),
        layers.Dropout(0.5),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=['accuracy']
    )
    return model

# Wrap the model in KerasClassifier
model = KerasClassifier(model=create_model, verbose=1)

# Define the hyperparameter grid
param_grid = {
    'learning_rate': [0.001, 0.0001],
    'dropout_rate': [0.25, 0.5],
    'dense_units': [512, 1012],
    'batch_size': [32, 64],
    'epochs': [20, 30]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)
grid_result = grid.fit(train_ds, val_ds)  # Replace with your training data

# Print the best results
print(f"Best Accuracy: {grid_result.best_score_} using {grid_result.best_params_}")

In [ ]:
data_augmentation = keras.Sequential(
    [layers.RandomRotation(0.2), layers.RandomZoom(0.15)]
)